In [27]:
import sys
sys.executable
import torch
import os
import sys
sys.path.append('../')
from models import *
import config
import numpy as np
from collections import defaultdict
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

In [2]:
in_path = '../benchmarks/ECB_rpi/'

In [5]:
with open( in_path + 'entity2id.txt') as nf:
    lines = [line.strip().split('\t') for line in nf.readlines()[1:]]
    ent2id = {line[0]:int(line[1]) for line in lines if 'events' in line[0]}
len(ent2id)

3591

In [11]:
with open(in_path + '/triples/eve_map.csv') as ef:
    lines = [line.strip().split(',') for line in ef.readlines() if line.strip().split(',')[1] in ent2id]
win_doc_dict = {line[0]:line[1] for line in lines}

In [15]:
event_ids_dict = ent2id
id2event_dict = {val:key for key, val in event_ids_dict.items()}
event_ids = event_ids_dict.values()
event2ind = {eve:i for i, eve in enumerate(event_ids_dict.keys())}
ind2event = {i:eve for eve, i in event2ind.items()}

In [17]:
con2 = config.Config()
con2.set_in_path(in_path)
con2.init()

In [20]:
gte = GraphTransE(con2)
gte.load_state_dict(torch.load('../result/GraphTransE.ckpt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
tfidf_tensor = sparse_tensor(gte.tf_idf_mat)
ent_emb_mat = gte.ent_char_weights(tfidf_tensor).cpu().data.numpy()
rel_emb_mat = gte.rel_embeddings.weight.data.numpy()

In [24]:
event_embeddings_2 = ent_emb_mat[list(event_ids), :]

In [29]:
event_embeddings_normed2 = np.array(event_embeddings_2)
event_embeddings_normed2 = event_embeddings_normed2/(np.linalg.norm(event_embeddings_normed2, axis=1).reshape((-1,1)))
eve_sim_mat2 = event_embeddings_normed2.dot(event_embeddings_normed2.T)
eve_adj_mat2 = csr_matrix(eve_sim_mat2 > 0.999)
eve_adj_mat2

<3591x3591 sparse matrix of type '<class 'numpy.bool_'>'
	with 77813 stored elements in Compressed Sparse Row format>

In [36]:
def get_connected_components(adjacency_mat):
    n_components, labels = connected_components(csgraph=adjacency_mat, directed=False, return_labels=True)
    clusters = defaultdict(list)
    for id_, lab in enumerate(labels):
        clusters[lab] += [id_]
    return clusters.values()

In [38]:
eve_clusters = get_connected_components(eve_adj_mat2)
eve_clusters_big = [clus for clus in eve_clusters if len(clus) >= 2]

In [52]:
eveid_clus_dict = {}
clus_num = 0
for clus in eve_clusters:
    for eve_ind in clus:
        eveid_clus_dict[ind2event[eve_ind]] = str(clus_num)
    clus_num+=1

In [53]:
cross_doc_dict = {}
for key, val in win_doc_dict.items():
    cross_doc_dict[key] = eveid_clus_dict[val]

In [55]:
with open('event_win_clus.txt', 'w') as tf:
    tf.write('\n'.join([','.join(item) for item in win_doc_dict.items()]))
with open('event_clusters.txt', 'w') as tf:
    tf.write('\n'.join([','.join(item) for item in cross_doc_dict.items()]))